In [1]:
import numpy as np
import pandas as pd

In [2]:
dataFilter = [['FullBiasNanos','FullBiasNanos ~= 0'],['ConstellationType','ConstellationType==1'],['State','bitand(State,2^0) & bitand(State,2^3)']]
gnssAnalyze = {
    "GnssClockErrors":'GnssClock Errors.' ,
    "GnssMeasurementErrors":'GnssMeasurement Errors.',
    "ApiPassFail":'' 
}

In [49]:
prFileName = '1.txt'
dirName = '/home/lyt/gnss_wifi/gnss/GPSdata/'
if not os.path.isdir("test_tranfer/outdoor/01"):
    os.makedirs("test_transfer/outdoor/01")
outputdir = 'test_transfer/outdoor/01'

In [4]:
f = open(dirName + prFileName)
datastring = f.read()
#datastring

In [5]:
f = open(dirName + prFileName)
datastring = f.read()
f = open(dirName + prFileName)
data = f.readlines()
f.close()
iDigits = datastring.find('Version')
version = datastring[iDigits+9:iDigits+17]
iDigits = datastring.find('Platform')
platform = datastring[iDigits+10:iDigits+15]

In [6]:
version, platform

('v2.0.0.1', '8.0.0')

In [7]:
rawlist = []
for i in range(len(data)):
    if data[i].find('Raw') != -1 and data[i].find('# Raw')==-1 :
        rawlist.append(i)
#print(rawlist) #12,22  37,46

In [8]:

head = data[5].split(',')[1:]
head[-1] = head[-1][:head[-1].find('\n')]

df2 = pd.DataFrame(columns=head)
for j in range(len(rawlist)):
    i = rawlist[j]
    if j == 0:
        time = 0
        df = pd.DataFrame(columns=head)
    ldata_f = []
    ldata = data[i].split(',')[1:]
    ldata[-1] = ldata[-1][:ldata[-1].find('\n')]
    for m in ldata:
        if m != '':
            ldata_f.append(float(m))
        else:
            ldata_f.append(m)
    #print(j)
    ldf = pd.DataFrame([ldata_f],columns=head)
    
    df = df.append(ldf,ignore_index=True)
    
    if i == rawlist[-1]:
        #df = df.sort_values(by=['Cn0DbHz'],ascending=False)
        #df = df.drop_duplicates('Svid')
        #df = df.reset_index()
        df = pd.concat([df],keys=['t'+ str(time)])
        t_end = time
        df = df.assign(allRxMills = (df.TimeNanos - df.FullBiasNanos)*1e-6)
        df_g = pd.concat([df_g,df])
        
    elif rawlist[j+1] -i >1:
        #df = df.sort_values(by=['Cn0DbHz'],ascending=False)
        #df = df.drop_duplicates('Svid')
        #df = df.reset_index()
        df = pd.concat([df],keys=['t'+ str(time)])
        df = df.assign(allRxMills = (df.TimeNanos - df.FullBiasNanos)*1e-6)
        if time == 0:
            df_g = pd.concat([df])
        else:
            df_g = pd.concat([df_g,df])
        time += 1
        df = pd.DataFrame(columns=head)
    
    


In [9]:
head

['ElapsedRealtimeMillis',
 'TimeNanos',
 'LeapSecond',
 'TimeUncertaintyNanos',
 'FullBiasNanos',
 'BiasNanos',
 'BiasUncertaintyNanos',
 'DriftNanosPerSecond',
 'DriftUncertaintyNanosPerSecond',
 'HardwareClockDiscontinuityCount',
 'Svid',
 'TimeOffsetNanos',
 'State',
 'ReceivedSvTimeNanos',
 'ReceivedSvTimeUncertaintyNanos',
 'Cn0DbHz',
 'PseudorangeRateMetersPerSecond',
 'PseudorangeRateUncertaintyMetersPerSecond',
 'AccumulatedDeltaRangeState',
 'AccumulatedDeltaRangeMeters',
 'AccumulatedDeltaRangeUncertaintyMeters',
 'CarrierFrequencyHz',
 'CarrierCycles',
 'CarrierPhase',
 'CarrierPhaseUncertainty',
 'MultipathIndicator',
 'SnrInDb',
 'ConstellationType',
 'AgcDb',
 'CarrierFrequencyHz']

In [10]:
df_g.keys()

Index(['ElapsedRealtimeMillis', 'TimeNanos', 'LeapSecond',
       'TimeUncertaintyNanos', 'FullBiasNanos', 'BiasNanos',
       'BiasUncertaintyNanos', 'DriftNanosPerSecond',
       'DriftUncertaintyNanosPerSecond', 'HardwareClockDiscontinuityCount',
       'Svid', 'TimeOffsetNanos', 'State', 'ReceivedSvTimeNanos',
       'ReceivedSvTimeUncertaintyNanos', 'Cn0DbHz',
       'PseudorangeRateMetersPerSecond',
       'PseudorangeRateUncertaintyMetersPerSecond',
       'AccumulatedDeltaRangeState', 'AccumulatedDeltaRangeMeters',
       'AccumulatedDeltaRangeUncertaintyMeters', 'CarrierFrequencyHz',
       'CarrierCycles', 'CarrierPhase', 'CarrierPhaseUncertainty',
       'MultipathIndicator', 'SnrInDb', 'ConstellationType', 'AgcDb',
       'CarrierFrequencyHz', 'allRxMills'],
      dtype='object')

In [11]:
#df_g['Cn0DbHz']['t'+str(0)].mean()

In [12]:
def JulianDay(time):
    y = time[0]
    m = time[1]
    d = time[2]
    h = time[3] + time[4]/60 + time[5]/3600;
    if m<=2: #index into months <=2
        m = m + 12
        y = y - 1
    return np.floor(365.25*y) + np.floor(30.6001*(m+1)) -15 + 1720996.5 + d + h/24

utcTable = [[1982, 1, 1, 0, 0, 0],
            [1982, 7, 1, 0, 0, 0],
            [1983, 7, 1, 0, 0, 0],
            [1985, 7, 1, 0, 0, 0],
            [1988, 1, 1, 0, 0, 0],
            [1990, 1, 1, 0, 0, 0],
            [1991, 1, 1, 0, 0, 0],
            [1992, 7, 1, 0, 0, 0],
            [1993, 7, 1, 0, 0, 0],
            [1994, 7, 1, 0, 0, 0],
            [1996, 1, 1, 0, 0, 0],
            [1997, 7, 1, 0, 0, 0],
            [1999, 1, 1, 0, 0, 0],
            [2006, 1, 1, 0, 0, 0],
            [2009, 1, 1, 0, 0, 0],
            [2012, 7, 1, 0, 0, 0],
            [2015, 7, 1, 0, 0, 0],
            [2017, 1, 1, 0, 0, 0]
         ]
tableJDays=[]
GPSEPOCHJD = 2444244.5
DAYSEC = 86400
for ut in utcTable:
    tableJDays.append(JulianDay(ut)- GPSEPOCHJD)
tableSeconds = np.array(tableJDays) * DAYSEC
jDays = []
for ut in utcTime:
    jDays = JulianDay(ut)- GPSEPOCHJD
timeSeconds = np.array(jDays)*DAYSEC
leapSecs= np.zeros(1)

np.sum(timeSeconds>=tableSeconds)

In [13]:
def LeapSeconds(utcTime):
    utcTable = [[1982, 1, 1, 0, 0, 0],
            [1982, 7, 1, 0, 0, 0],
            [1983, 7, 1, 0, 0, 0],
            [1985, 7, 1, 0, 0, 0],
            [1988, 1, 1, 0, 0, 0],
            [1990, 1, 1, 0, 0, 0],
            [1991, 1, 1, 0, 0, 0],
            [1992, 7, 1, 0, 0, 0],
            [1993, 7, 1, 0, 0, 0],
            [1994, 7, 1, 0, 0, 0],
            [1996, 1, 1, 0, 0, 0],
            [1997, 7, 1, 0, 0, 0],
            [1999, 1, 1, 0, 0, 0],
            [2006, 1, 1, 0, 0, 0],
            [2009, 1, 1, 0, 0, 0],
            [2012, 7, 1, 0, 0, 0],
            [2015, 7, 1, 0, 0, 0],
            [2017, 1, 1, 0, 0, 0]
         ]
    tableJDays=[]
    GPSEPOCHJD = 2444244.5
    DAYSEC = 86400
    for ut in utcTable:
        tableJDays.append(JulianDay(ut)- GPSEPOCHJD)
    tableSeconds = np.array(tableJDays) * DAYSEC
    jDays = []
    for ut in utcTime:
        jDays = JulianDay(ut)- GPSEPOCHJD
    timeSeconds = np.array(jDays)*DAYSEC
    leapSecs= np.zeros(1)
    return np.sum(timeSeconds>=tableSeconds)

In [14]:
def Fct2Ymdhms(svtest):
    
    HOURSEC = 3600
    MINSEC = 60
    monthDays=[31,28,31,30,31,30,31,31,30,31,30,31]
    DAYSEC = 86400
    days = np.array(np.floor(svtest/DAYSEC)+6)
    years = np.zeros([days.size])+1980
    leap = np.ones([days.size])
    time = np.zeros([days.size,6])
    while((days > leap+365).any()):
        I = np.where(days > (leap+365),1,0)
        days = days - I * (leap + 365)
        years =years + I;
        leap = leap*(1-I) + np.where(years%4 == 0,1,0)
    time[:,0] = years
    
    for i in range(days.size):
        month = 1
        if (years[i]%4 == 0):  #This works from 1901 till 2099
            monthDays[1]=29 #Make February have 29 days in a leap year
        else:
            monthDays[1]=28
        while (days[i] > monthDays[month]):
            days[i] = days[i]-monthDays[month-1]
            month = month+1
        time[i,1] = month
    
    time[:,2] = days
    sinceMidnightSeconds = svtest % DAYSEC
    time[:,3] = np.floor(sinceMidnightSeconds/HOURSEC)
    lastHourSeconds = sinceMidnightSeconds % HOURSEC
    time[:,4] = np.floor(lastHourSeconds/MINSEC)
    time[:,5] = lastHourSeconds%MINSEC
    return time

In [15]:
def gps2utc(allRxMills):
    svtest = 1e-3*np.array(allRxMills)
    time = Fct2Ymdhms(svtest)
    ls = LeapSeconds(time)
    timeMLs = Fct2Ymdhms(svtest-ls)
    ls1 = LeapSeconds(timeMLs)
    if (ls1==ls):
        utcTime = timeMLs
    else:
        utcTime = Fct2Ymdhms(fctSeconds-ls1)
    return utcTime

In [16]:
def gps2glo(t):
    svtest = 1e-3*np.array(t)
    time = Fct2Ymdhms(svtest)
    ls = LeapSeconds(time)
    timeMLs = Fct2Ymdhms(svtest-ls)
    ls1 = LeapSeconds(timeMLs)
    if (ls1==ls):
        gloTime = svtest-ls+10800
    else:
        gloTime = svtest-ls1+10800
    return gloTime

In [17]:
def utc2gps(utcTime):
    gpsTime = []
    fctSeconds = []
    HOURSEC = 3600
    MINSEC = 60
    GPSEPOCHJD = 2444244.5    
    daysSinceEpoch = np.floor(JulianDay(utcTime) - GPSEPOCHJD);
    gpsWeek = np.fix(daysSinceEpoch/7)
    dayofweek = np.remainder(daysSinceEpoch,7)
    DAYSEC = 86400
    gpsSeconds = dayofweek*DAYSEC + utcTime[3]*HOURSEC + utcTime[4]*MINSEC + utcTime[5]
    WEEKSEC = 604800
    gpsWeek = gpsWeek + np.fix(gpsSeconds/WEEKSEC)
    gpsSeconds = np.remainder(gpsSeconds,WEEKSEC)
    leapSecs = LeapSeconds([utcTime])
    fctSeconds = gpsWeek*WEEKSEC + gpsSeconds# + leapSecs
    gpsWeek = np.fix(fctSeconds/WEEKSEC)
    if (gpsWeek==0):
        iZ = 1
    else:
        iZ = 0
    if iZ:
        gpsSeconds = fctSeconds
    else:
        gpsSeconds = np.remainder(fctSeconds,gpsWeek*WEEKSEC)
    gpsTime=[gpsWeek,gpsSeconds]

    return gpsTime,fctSeconds

In [18]:
utcTime = gps2utc(df_g.allRxMills.t0[0])
utcTime

array([[2020.       ,    7.       ,    6.       ,    7.       ,
          55.       ,   10.4491868]])

In [19]:
gpsTime, F= utc2gps(utcTime[0])
gpsTime,F

([2113.0, 114910.44918680191], 1278057310.4491868)

In [20]:
#utcTime = gps2utc(df_g["allRxMills"]["t"+str(t_end)].iloc[-1])

In [21]:
yearNumber4Digit = utcTime[0][0];
yearNumber2Digit = utcTime[0][0]%100;


jDay = JulianDay([utcTime[0][0],utcTime[0][1],utcTime[0][2],0,0,0])
jDayJan1 = JulianDay([utcTime[0][0],1,1,0,0,0])
dayNumber = jDay - jDayJan1 + 1

In [22]:
hourlyZFile = 'hour{:03.0f}0.{:02.0f}n.Z'.format(dayNumber,yearNumber2Digit)

In [23]:
hourlyZFile

'hour1880.20n.Z'

In [24]:
ephFilename = hourlyZFile.split('.Z')[0]

In [25]:
ephFilename

'hour1880.20n'

In [26]:
import os 
from ftplib import FTP
import zlib
if not os.path.isdir("../gnss/GPSephermise"):
    os.makedirs("../gnss/GPSephermise")

if not os.path.isfile("../gnss/GPSephermise/"+ephFilename):
    Ephdir = "./GPSephermise/"
    ftpSiteName = 'cddis.gsfc.nasa.gov'
    hourlyDir = '/gnss/data/hourly/%4d/%03d/' + str(yearNumber4Digit) + str(dayNumber)
    try:
        ftp = FTP(ftpSiteName)
        ftp.login()
        ftp.cwd('/gnss/data/hourly/{:4.0f}/{:03.0f}'.format(yearNumber4Digit,dayNumber))
        ftp.retrbinary("RETR " + hourlyZFile, open(Ephdir+hourlyZFile, 'wb').write)
        os.system( "uncompress " + Ephdir +hourlyZFile)

    except:
        print('\nAutomatic ftp download failed.\n')
        print('Please go to this ftp, ftp://cddis.gsfc.nasa.gov/ \n')
        print('and get this file:{:s}{:s} \n'.format(hourlyDir,hourlyZFile))
        print('Extract contents to the directory with your pseudorange log file:\n')
        print('{:s}\n'.format(dirName))
        print('To unzip the *.Z file, see http://www.gzip.org/\n')
        print('Then run this function again, it will read from that directory\n')
allGpsEph = open("../gnss/GPSephermise/"+ephFilename,"r")
rinex = allGpsEph.read().splitlines()

In [27]:
class Iono():
    def __init__(self,alpha,beta):
        self.alpha = alpha
        self.beta = beta

In [28]:
for i in range(len(rinex)):
    if rinex[i].find("END OF HEADER") != -1:
        numHdrLines = i+1
    if rinex[i].find("ION ALPHA") != -1:
        alpha = rinex[i][:rinex[i].find("ION ALPHA") -1].split()
        for i in range (len(alpha)):
            alpha[i] = float(alpha[i].replace('D','e'))
    if rinex[i].find("ION BETA") != -1:
        beta = rinex[i][:rinex[i].find("ION BETA") -1].split()
        for i in range (len(beta)):
            beta[i] = float(beta[i].replace('D','e'))
iono = Iono(alpha,beta)
numEph = (len(rinex)-numHdrLines)/8

In [29]:
gpsEphs=[]

In [30]:
head = ['PRN','Toc','af0','af1','af2','IODE','Crs','Delta_n','M0','Cuc','e','Cus','Asqrt','Toe','Cic','OMEGA','Cis','i0','Crc','omega'
        ,'OMEGA_DOT','IDOT','codeL2','GPS_Week','L2Pdata','accuracy','health','TGD','IODC','ttx','Fit_interval']
allGpsEphData = pd.DataFrame(columns=head)

In [31]:
for i in range(int(numEph)):
    gpsEphs=[]
    j = 0
    line = rinex[(i+1)*8+j].replace("D","e")
    line = line.replace("-0."," -0.")
    line = line.split()
    gpsEphs.append(float(line[0]))
    year = float(line[1])
    if year < 80:
       year = 2000+year
    else:
       year = 1900+year
    month = float(line[2])
    day = float(line[3])
    hour = float(line[4])
    minute = float(line[5])
    second = float(line[6])
    gpsTime,F = utc2gps([year,month,day,hour,minute,second])
    gpsEphs.append(gpsTime[1])
    gpsEphs.append(float(line[7]))
    gpsEphs.append(float(line[8]))
    gpsEphs.append(float(line[9]))
    
    j = 1
    line = rinex[(i+1)*8+j].replace("D","e")
    line = line.replace("-0."," -0.")
    line = line.split()
    gpsEphs.append(float(line[0]))
    gpsEphs.append(float(line[1]))
    gpsEphs.append(float(line[2]))
    gpsEphs.append(float(line[3]))
    
    j = 2
    line = rinex[(i+1)*8+j].replace("D","e")
    line = line.replace("-0."," -0.")
    line = line.split()
    gpsEphs.append(float(line[0]))
    gpsEphs.append(float(line[1]))
    gpsEphs.append(float(line[2]))
    gpsEphs.append(float(line[3]))
    
    j = 3
    line = rinex[(i+1)*8+j].replace("D","e")
    line = line.replace("-0."," -0.")
    line = line.split()
    gpsEphs.append(float(line[0]))
    gpsEphs.append(float(line[1]))
    gpsEphs.append(float(line[2]))
    gpsEphs.append(float(line[3]))
    
    j = 4
    line = rinex[(i+1)*8+j].replace("D","e")
    line = line.replace("-0."," -0.")
    line = line.split()
    gpsEphs.append(float(line[0]))
    gpsEphs.append(float(line[1]))
    gpsEphs.append(float(line[2]))
    gpsEphs.append(float(line[3]))
    
    j = 5
    line = rinex[(i+1)*8+j].replace("D","e")
    line = line.replace("-0."," -0.")
    line = line.split()
    gpsEphs.append(float(line[0]))
    gpsEphs.append(float(line[1]))
    gpsEphs.append(float(line[2]))
    gpsEphs.append(float(line[3]))
    
    j = 6
    line = rinex[(i+1)*8+j].replace("D","e")
    line = line.replace("-0."," -0.")
    line = line.split()
    gpsEphs.append(float(line[0]))
    gpsEphs.append(float(line[1]))
    gpsEphs.append(float(line[2]))
    gpsEphs.append(float(line[3]))
    
    j = 7
    line = rinex[(i+1)*8+j].replace("D","e")
    line = line.replace("-0."," -0.")
    line = line.split()
    gpsEphs.append(float(line[0]))
    gpsEphs.append(float(line[1]))
    data = pd.DataFrame([gpsEphs],columns=head)
    allGpsEphData = allGpsEphData.append(data,ignore_index=True)

In [32]:
#allGpsEphData

In [33]:
len(gpsEphs)

31

In [34]:
def RxNanos(ConstellationType,FullBiasNanos,TimeNanos):
    WEEKSEC = 604800
    DAYSEC = 86400
    GLO = ConstellationType == 3
    BEI = ConstellationType == 5
    GPSGLA = np.logical_not(GLO+BEI)
    rxnano = np.zeros(ConstellationType.size)
    dayNumber = np.floor( -FullBiasNanos[GLO] * 1e-9/DAYSEC)
    weekNumber = np.floor( -FullBiasNanos[BEI+GPSGLA] * 1e-9/WEEKSEC)
    dayNumberNanos = dayNumber * DAYSEC*1e9
    weekNumberNanos = weekNumber * WEEKSEC*1e9
    rxgpstime = TimeNanos - FullBiasNanos[0]
    if any(GLO):
        glorx = gps2glo(rxgpstime[GLO]*1e-6)*1e9 - dayNumberNanos
        rxnano[GLO] = glorx
    others = rxgpstime[BEI+GPSGLA] - weekNumberNanos
    rxnano[BEI+GPSGLA] = others
    if any(BEI):
        rxnano[BEI]-=14*1e9
    return rxnano

In [35]:
WEEKSEC = 604800
test = df_g
iTowUnc = test.ReceivedSvTimeUncertaintyNanos > 500
iPrrUnc = test.PseudorangeRateUncertaintyMetersPerSecond > 10
df_g = df_g[df_g['ReceivedSvTimeUncertaintyNanos']<=500]
df_g = df_g[df_g['PseudorangeRateUncertaintyMetersPerSecond']<=10]
df_g = df_g.assign(tRxNanos = RxNanos(np.array(df_g.ConstellationType),np.array(df_g.FullBiasNanos),np.array(df_g.TimeNanos)))
df_g = df_g.assign(tRxSeconds  = (df_g.tRxNanos- df_g.TimeOffsetNanos - df_g.BiasNanos)*1e-9)
df_g = df_g.assign(tTxSeconds  = (df_g.ReceivedSvTimeNanos)*1e-9)
prSeconds  = np.array(df_g.tRxSeconds - df_g.tTxSeconds)
iRollover = np.array(prSeconds > WEEKSEC/2)
prS = prSeconds[iRollover]
delS = np.round(prS/WEEKSEC)*WEEKSEC
prS = prS - delS

maxBiasSeconds = 10; 
if any(prS>maxBiasSeconds):
    print('Failed to correct week rollover\n')
else:
    prSeconds[iRollover] = prS; 
    df_g.tRxSeconds[iRollover] = df_g.tRxSeconds[iRollover] - delS;
    print('Corrected week rollover\n')

    

LIGHTSPEED = 2.99792458e8
df_g = df_g.assign(PrM  = prSeconds * LIGHTSPEED)
df_g = df_g.assign(PrSigmaM = (df_g.ReceivedSvTimeUncertaintyNanos)*1e-9*LIGHTSPEED)




Corrected week rollover



In [36]:
test = df_g[df_g['ConstellationType']==1]
test = test.assign(weekNum = np.floor(test.allRxMills *1e-3 / WEEKSEC))

In [37]:
test.allRxMills.t100[4],test[test['Svid']==5]['allRxMills']['t100']

(1278057428449.186,
 4    1.278057e+12
 Name: allRxMills, dtype: float64)

In [38]:
svid = np.array(test['Svid']['t'+str(100)])
svid

array([ 5.,  7., 12., 13., 15., 25.])

In [52]:
outputdir+"/{:02d}.txt".format(t)

'test_transfer/outdoor/01/00.txt'

In [84]:
for t in range(t_end+1):
    #t = 0
    #fp = open(outputdir+"/{:02d}.txt".format(t),"w")
    svid = np.array(test['Svid']['t'+str(t)])
    iSv = []
    GpsEph = pd.DataFrame(columns=head)
    for i in range(len(svid)):
        sv_id = svid[i]    
        ephThisSv = allGpsEphData[allGpsEphData['PRN']==sv_id]
        fitIntervalHours = np.array(ephThisSv.Fit_interval)
        fitIntervalHours[fitIntervalHours == 0]=4
        fitIntervalHours
        fctToe = np.array(ephThisSv.GPS_Week*WEEKSEC + ephThisSv.Toe)
        ageSeconds = min(abs(fctToe-float(test[test['Svid']==sv_id]['allRxMills']['t'+str(t)]*1e-3)))
        iMin = np.argmin(abs(fctToe-float(test[test['Svid']==sv_id]['allRxMills']['t'+str(t)]*1e-3)))
        if ageSeconds < (fitIntervalHours[iMin]/2)*3600:
            numEph = numEph+1
            GpsEph = GpsEph.append(ephThisSv.iloc[iMin],ignore_index=True)
            iSv.append(i)

    svid = np.array(test['Svid']['t'+str(t)].iloc[iSv])
    numSvs = len(svid)
    #if numSvs < 4:
    #    continue
    prM = np.array(test['PrM']['t'+str(t)].iloc[iSv])
    prSigmaM = np.array(test['PrSigmaM']['t'+str(t)].iloc[iSv])
    prrMps = np.array(test['PseudorangeRateMetersPerSecond']['t'+str(t)].iloc[iSv])
    prrSigmaMps = np.array(test['PseudorangeRateUncertaintyMetersPerSecond']['t'+str(t)].iloc[iSv])
    tRxweekNum = np.array(test['weekNum']['t'+str(t)].iloc[iSv])
    tTxweekNum = tRxweekNum
    tTxSeconds = np.array(test['tTxSeconds']['t'+str(t)].iloc[iSv])
    #tRx = np.concatenate((np.array( [test['weekNum']['t'+str(t)].iloc[iSv]]).transpose(),np.array([test['tRxSeconds']['t'+str(t)].iloc[iSv]]).transpose()),axis=1)
    tk = tTxSeconds - GpsEph.Toe
    A = np.square(np.array(GpsEph.Asqrt))
    mu = 3.986005e14
    n0 = np.sqrt(mu / np.power(A,3))
    n = n0 + GpsEph.Delta_n
    Mk = GpsEph.M0 + n * tk

    err = np.array([1]);

    ek=Mk
    iterCount = 0;
    maxIterCount = 20;
    while any(abs(err) > 1e-8):# & iterCount < maxIterCount:
      err = ek - Mk - (GpsEph.e*np.sin(ek));
      ek = ek - err;
      iterCount = iterCount + 1

    dt = tTxSeconds - GpsEph.Toc
    FREL = -4.442807633e-10
    dtsvS = GpsEph.af0 + GpsEph.af1*dt + GpsEph.af2*np.square(dt)  + FREL*GpsEph.e*GpsEph.Asqrt*np.sin(ek) -GpsEph.TGD
    ttx = tTxSeconds - dtsvS
    fitIntervalHours = GpsEph.Fit_interval
    fitIntervalHours[fitIntervalHours == 0] = 2
    fitIntervalSeconds = fitIntervalHours*3600
    tk =(tTxweekNum-GpsEph.GPS_Week)*WEEKSEC + (ttx-GpsEph.Toe)
    Mk = GpsEph.M0 + n * tk
    err = np.array([1]);

    ek=Mk
    iterCount = 0;
    maxIterCount = 20;
    while any(abs(err) > 1e-8):# & iterCount < maxIterCount:
      err = ek - Mk - (GpsEph.e*np.sin(ek));
      ek = ek - err;
      iterCount = iterCount + 1

    vk=np.arctan2(np.sqrt(1-np.square(GpsEph.e))*np.sin(ek)/(1-GpsEph.e*np.cos(ek)),(np.cos(ek)-GpsEph.e)/(1-GpsEph.e*np.cos(ek)))
    Phik=vk + GpsEph.omega

    sin_2Phik=np.sin(2*Phik)
    cos_2Phik=np.cos(2*Phik)

    duk = GpsEph.Cus*sin_2Phik + GpsEph.Cuc*cos_2Phik
    drk = GpsEph.Crc*cos_2Phik + GpsEph.Crs*sin_2Phik
    dik = GpsEph.Cic*cos_2Phik + GpsEph.Cis*sin_2Phik

    uk = Phik + duk
    rk = A*((1-np.square(GpsEph.e))/(1+GpsEph.e*np.cos(vk))) + drk
    ik = GpsEph.i0 + GpsEph.IDOT*tk + dik

    sin_uk=np.sin(uk)
    cos_uk=np.cos(uk)
    xkp = rk*cos_uk
    ykp = rk*sin_uk

    WE = 7.2921151467e-5
    Wk  = GpsEph.OMEGA + (GpsEph.OMEGA_DOT - WE)*tk - WE*GpsEph.Toe

    sin_Wk = np.sin(Wk)
    cos_Wk = np.cos(Wk)

    sin_ik=np.sin(ik)
    cos_ik=np.cos(ik)

    a = np.array([xkp*cos_Wk-ykp*cos_ik*sin_Wk])
    b= np.array([xkp*sin_Wk+ykp*cos_ik*cos_Wk])
    c = np.array([ykp*sin_ik])

    d = np.concatenate((a.transpose(),b.transpose()),axis=1)
    d = np.concatenate((d,c.transpose()),axis=1)
    d = np.concatenate((d,np.array([prM]).transpose()),axis=1)
    np.savetxt(outputdir+"/{:02d}.txt".format(t),d)
    #fp.close()

In [85]:
np.loadtxt(outputdir+"/00.txt",delimiter=" ")

array([[    59942.9218576 ,  23975236.73586827,  11070502.82859027,
         20989574.42069478],
       [ 10434601.62025295,  23420815.01705759,   6992155.38830839,
         23135026.85591651],
       [ -9344213.3200727 ,  20541207.27183969, -14044732.94811807,
         23500451.97466699],
       [ 15622227.97970926,  15462465.43570905,  14857540.12067312,
         24628421.10129827]])

In [77]:
np.loadtxt("transfer_gps/outdoor/01/00.txt",delimiter=",")

array([[    59942.9218576 ,  23975236.73586827,  11070502.82859027,
         20989574.42069478],
       [ 10434601.62025295,  23420815.01705759,   6992155.38830839,
         23135026.85591651],
       [ -9344213.3200727 ,  20541207.27183969, -14044732.94811807,
         23500451.97466699],
       [ 15622227.97970926,  15462465.43570905,  14857540.12067312,
         24628421.10129827]])

In [407]:
rinex

['     2              NAVIGATION DATA                         RINEX VERSION / TYPE',
 'CCRINEXN V1.6.0 UX  CDDIS               01-JUL-20 23:30     PGM / RUN BY / DATE ',
 'IGS BROADCAST EPHEMERIS FILE                                COMMENT             ',
 '    0.4657D-08  0.1490D-07 -0.5960D-07 -0.1192D-06          ION ALPHA           ',
 '    0.8192D+05  0.8192D+05 -0.6554D+05 -0.5243D+06          ION BETA            ',
 '    0.186264514923D-08 0.621724893790D-14   503808     2112 DELTA-UTC: A0,A1,T,W',
 '    18                                                      LEAP SECONDS        ',
 '                                                            END OF HEADER       ',
 ' 1 20  7  1  0  0  0.0 0.193901360035D-04 0.613908923697D-11 0.000000000000D+00',
 '    0.400000000000D+01-0.493750000000D+02 0.395766485263D-08-0.125610152048D+01',
 '   -0.248849391937D-05 0.100110108033D-01 0.102519989014D-04 0.515371048737D+04',
 '    0.259200000000D+06 0.227242708206D-06 0.244838164956D+01 0.335